<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/Insight-Sogang-Univ/insight-14th/blob/main/advanced/template/session05/assignment_languagemodel.ipynb" target="_parent">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
  </td>
</table>


# 🔍 이론 문제

### 1. SLM(통계적 언어모델)의 한계에 대해 설명하세요.

**[1번 문제 정답]**

편향, 제한된 일반화, 할루시네이션, 성능과 용량의 한계
[정답]


### 2. BERT에 대한 설명이 아닌 것은?

a) 사전 학습 모델

b) 양방향 문맥 이해

c) 트랜스포머의 디코더를 쌓아 올린 구조

d) 이후 RoBERTa와 ALBERT로 발전

**[2번 문제 정답]**

c
[정답]

### 3. BERT와 GPT의 학습 방식 차이를 간단히 설명하세요.

**[3번 문제 정답]**

BERT는 트랜스포머의 인코더를 쌓아 올린 구조, MLM, NSP 사전학습 과정을 거친다.
GPT는 트랜스포머의 디코더를 쌓아 올린 구조,
Masked Attention으로 동작하고 BERT 문장 전체를 보고 빈칸을 맞추고 GPT는 앞부분 단어만 보고 다음 단어를 생성한다
[정답]

### 4. LLM이 지닌 한계와 RAG의 필요성에 대해 간단히 서술하세요.

**[4번 문제 정답]**

환각, 새로운 정보를 반영하려면 전체 모델 재학습해야 되는데 이러면 업데이트 비용이 막대하다. 그래서 RAG로 관련된 정보를 검색하여 먼저 찾고 이를 LLM에 넣어 최종 답변을 생성한다\
[정답]

### 5. LLM을 경량화한 모델인 sLM을 만들기 위해 사용되는 대표적인 기술 3가지를 적으세요.(단답형)

**[5번 문제 정답]**

가지치기, 양자화, 지식 증류 \
[정답]

# ✍ 언어 모델 과제

### 🧮 GPU 사용하기

In [1]:
import torch
import tensorflow as tf

# PyTorch GPU 상태
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ [PyTorch] Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")


# TensorFlow GPU 상태
print("\n✅ [TensorFlow] GPU Devices:")
print(tf.config.list_physical_devices('GPU'))

✅ [PyTorch] Using device: cpu

✅ [TensorFlow] GPU Devices:
[]


# 🔍 1. BERT

**BERT (Bidirectional Encoder Representations from Transformers)**
- 문장 분류(스팸 메일 탐지), 질의응답 시스템(챗봇, 검색 엔진), 번역(다국어 지원), 텍스트 요약(뉴스 요약) 등 다양한 NLP 작업에 사용
- 문맥을 양방향으로 이해하여 텍스트의 의미를 정밀하게 파악하며, 사전 학습된 모델을 기반으로 빠르게 응용 가능

### 1-1. 모델과 tokenizer 초기화
- tokenizer를 통해 문장을 토큰으로 나누고, 이를 정수 인덱스로 변환하여 모델이 이해할 수 있도록 함

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 tokenizer 초기화
tokenizer = BertTokenizer.from_pretrained('klue/bert-base') # base : 모델크기, (uncased : 소문자 학습)
model = BertForMaskedLM.from_pretrained('klue/bert-base') # base : 모델크기, (uncased : 소문자 학습)
# tokenizer : raw text를 개별 토큰으로 분리(Wordpiece). 토크나이저 사전에 따라 고유한 정수 인덱스(고정값)를 매핑함.

# tokenizer를 통해 생성된 정수 인덱스 : 텍스트를 모델이 읽을 수 있도록 숫자화
# Embedding vector : 단어의 의미적, 문맥적 특성을 모델링

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 1-2. 입력 텍스트 준비 및 토큰화
- [MASK] 토큰을 삽입한 문장을 설정하고, `tokenizer.tokenize()`로 텍스트 토큰화
- [MASK]를 활용하여 문맥 속에서 특정 단어를 추론하는 상황을 만들고, 이를 모델이 학습한 패턴과 비교하도록 함


In [ ]:
# 테스트할 문장
text = "[CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]"

# 문장을 토큰으로 변환
tokenized_text = tokenizer.tokenize(text) # text(문장)을 토큰으로 변환해 tokenized_text에 저장합시다!
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰을 index로 변환하는 코드!

print(tokenized_text) # 토큰이 출력됩니다.
print(indexed_tokens) # 인덱스가 출력됩니다.


['[CLS]', '열', '##여', '##덟', ',', '우리', '##는', '서로', '##의', '이름', '##을', '처음', '불렀', '##다', '.', '그리고', '스물', '하나', ',', '우린', '[MASK]', '을', '했', '##다', '.', '[SEP]']
[2, 1432, 2173, 3542, 16, 3616, 2259, 4084, 2079, 3934, 2069, 3790, 6895, 2062, 18, 3673, 10514, 3657, 16, 8983, 4, 1498, 1902, 2062, 18, 3]


### 1-3. MASK된 위치 확인
- [MASK] 토큰의 위치를 탐지해 해당 위치에서 모델이 단어를 예측하도록 지정합니다.


In [ ]:
# 마스킹된 위치 찾기
masked_index = tokenized_text.index("[MASK]") # 토큰화된 결과물에서 index 메서드를 통해 [MASK]의 위치를 확인해봅시다!
print(masked_index)

20


### 1-4. 정수화된 토큰 인덱스를 tensor로 변환 후 모델 예측 실행
- 모델은 **텐서 데이터 구조**를 필요로 하므로, **입력값을 변환**하여 적합한 형식으로 만듦.
   - `torch.tensor(입력값 리스트 또는 입력값 ndarray)`를 통해 tensor 구조로 변환 가능!
- **역전파를 비활성화**하고, 모델이 **각 토큰에 대해 가능한 모든 단어 점수 출력**

In [ ]:
# 토큰화된 텍스트 -> 정수인덱스(ID) -> Pytorch 텐서
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰을 index로 변환하는 코드!
tokens_tensor = torch.tensor([indexed_tokens]) # 입력값(토큰 인덱스 리스트)을 tensor 구조로 변환해봅시다!

# 모델에 토큰 텐서를 전달하고 예측 실행
with torch.no_grad(): # 가중치 업데이트 없으므로 자동미분연산은 끔
    outputs = model(tokens_tensor) # 예측. 모델은 각 토큰위치에 대한 예측 결과를 반환.
    predictions = outputs[0] # logits (각 토큰위치에서 가능한 모든 토큰들의 원시점수)
print(predictions)

tensor([[[ -6.0337,   4.5526,  -5.6312,  ...,  -7.3959,  -7.4220,  -5.8337],
         [ -6.1762,   4.7585,  -7.3973,  ...,  -7.6209, -12.2124,  -6.0998],
         [ -7.4194,   3.9148,  -6.1517,  ...,  -6.8162,  -9.5421,  -3.0473],
         ...,
         [ -7.3761,   8.6972,  -5.4904,  ...,  -8.7202,  -9.1403,  -5.8566],
         [ -5.6347,  10.3884,  -4.3374,  ...,  -8.6900,  -7.4171,  -3.5043],
         [ -5.6652,  10.3407,  -4.4099,  ...,  -8.7521,  -7.4286,  -3.6681]]])


### 1-5. MASK된 토큰 예측
- [MASK] 위치에서 가장 높은 점수를 받은 토큰의 인덱스를 추출하고, 이를 단어로 변환

In [ ]:
# 예측된 토큰 확인
predicted_index = torch.argmax(predictions[0, masked_index]).item() # masked_index에 들어갈 것으로 가장 확률이 높은 인덱스
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

# 예측된 토큰으로 마스크 채우기
tokenized_text[masked_index] = predicted_token
# 토큰화된 텍스트를 다시 문자열로 변환
filled_text = tokenizer.convert_tokens_to_string(tokenized_text[1:-1])

print("Original:", text)
print("Masked:", tokenized_text)
print("Predicted token:", predicted_token) ; print()

print("Filled sentence:", filled_text)

Original: [CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]
Masked: ['[CLS]', '열', '##여', '##덟', ',', '우리', '##는', '서로', '##의', '이름', '##을', '처음', '불렀', '##다', '.', '그리고', '스물', '하나', ',', '우린', '사랑', '을', '했', '##다', '.', '[SEP]']
Predicted token: 사랑

Filled sentence: 열여덟 , 우리는 서로의 이름을 처음 불렀다 . 그리고 스물 하나 , 우린 사랑 을 했다 .



### 1-6. MASK된 문장 복원
- 예측된 토큰으로 [MASK]를 대체하여 완성된 문장 생성

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = BertForMaskedLM.from_pretrained('klue/bert-base')

def fill_mask(input_text):
    # 텍스트를 토큰으로 변환
    tokenized_text = tokenizer.tokenize(input_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # 마스킹된 위치 찾기
    masked_index = tokenized_text.index("[MASK]") # 토큰화된 텍스트에서 마스킹된 위치를 찾아봅시다!

    # 토큰화된 텍스트 -> 정수인덱스(ID) -> Pytorch 텐서
    tokens_tensor = torch.tensor([indexed_tokens])

    # 모델에 토큰 텐서를 전달하고 예측 수행
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # 예측된 토큰 확인
    predicted_index = torch.argmax(predictions[0, masked_index]).item() # 위 코드 참고! masked_index에 들어갈 것으로 가장 확률이 높은 인덱스
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

    # 마스크를 채운 문장 반환
    tokenized_text[masked_index] = predicted_token # 토큰화된 텍스트의 마스킹된 위치(인덱스 사용)를 예측된 토큰으로 채워봅시다!
    return tokenizer.convert_tokens_to_string(tokenized_text[1:-1])

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


예시 문장 만들어서 MLM을 사용해 보세요!

In [ ]:
# 예시 1: "[CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]"
# 예시 2: "[CLS] 윤해민은 INSGIHT 학회를 정말 사랑한다. 그는 INSIGHT 학회를 위해 [MASK]를 했다.[SEP]"
# 이 예시를 참고해서 [CLS](맨 앞), [MASK](채울 부분), [SEP](맨 뒤)를 추가해서 여러분만의 예시 문장 만들어주세요
input_text = "[CLS] 너가 제일 좋아하는 동물은 [MASK]이다.[SEP]"
filled_text = fill_mask(input_text)
print("Filled sentence:", filled_text)

Filled sentence: 너가 제일 좋아하는 동물은 사자 이다 .


# 🔍 2. GPT, RAG, Langchain

## 2-1 라이브러리 설치

langchain 관련 라이브러리와, pdf 문서를 읽을수 있는 pypdf 라이브러리를 설치할게요!

In [2]:
# ⬇️ Colab에서 한 번만 실행
%pip install -qU "langchain>=0.3" langchain-community langchain-openai langgraph chromadb pypdf tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 31.3 MB/s eta 0:0

## 2-2. 키값 설정

In [ ]:
##################################################################
### 과제 제출 시 이 코드 셀은 삭제하고 저장 후, add, commit, push 해주세요 ###
##################################################################
import os

os.environ["OPENAI_API_KEY"] = "-"

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "-"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "practice_langgraph"



## 2-3 pdf 문서 선택 (로드)


여러분들이 활용하고 싶은 아무 pdf 문서를 선택해보세요!

In [ ]:
from google.colab import files
import os, re, unicodedata

os.makedirs("/content/pdfs", exist_ok=True)
print("📥 업로드할 PDF 파일을 선택하세요 (여러 개 가능).")
uploaded = files.upload()

name_map = {}
for i, (orig_name, data) in enumerate(uploaded.items(), 1):
    nf = unicodedata.normalize("NFC", orig_name)
    base, ext = os.path.splitext(nf)
    # 한글/영문/숫자/밑줄/대시/점만 남기고 나머지는 _
    safe_base = re.sub(r"[^\w\u3131-\u318E\uAC00-\uD7A3.\-]+", "_", base).strip("._'‘’“”")
    # 파일명 너무 길면 잘라냄 (최대 ~140자 정도로)
    safe_base = safe_base[:120] if len(safe_base) > 120 else safe_base
    new_name = f"doc_{i}_{safe_base}{ext.lower()}"
    with open(os.path.join("/content/pdfs", new_name), "wb") as f:
        f.write(data)
    name_map[new_name] = orig_name

print("✅ 저장 완료 (Colab 파일명 → 원본명 매핑):")
for k, v in name_map.items():
    print(f"  {k}  <=  {v}")


📥 업로드할 PDF 파일을 선택하세요 (여러 개 가능).


Saving 심청전 (현대문).pdf to 심청전 (현대문).pdf
✅ 저장 완료 (Colab 파일명 → 원본명 매핑):
  doc_1_심청전_현대문.pdf  <=  심청전 (현대문).pdf


## 2-4 문서 분할 후 벡터 DB에 저장
로딩한 문서를 분할 할 때,
- chunk_size는 `900`
- chunk_overlap은 `150`
- separators(구분자)는 `["\n\n", "\n", " ", ""]`
으로 설정하여 청킹해주세요!

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import glob

# 1) PDF → Documents
pdf_paths = sorted(glob.glob("/content/pdfs/*.pdf"))
docs = []
for p in pdf_paths:
    try:
        loader = PyPDFLoader(p)
        docs.extend(loader.load())
    except Exception as e:
        print(f"⚠️ 로딩 실패: {p} -> {e}")

print(f"총 문서 조각(페이지 기준): {len(docs)}")

# 2) 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1000, # 청크 사이즈: 문서를 몇 개의 토큰 단위로 나눌 것인지를 정합니다.
    chunk_overlap= 200, # 청크 오버랩: 분할된 끝 부분에서 맥락이 이어질 수 있도록 일부를 겹쳐서(overlap) 분할하는 것이 일반적입니다.
    separators= ["\n\n", "\n", " ", ""] # 구분자: 엔터, 공백 기준으로 나눔
)
chunks = text_splitter.split_documents(docs)
print(f"총 청크 수: {len(chunks)}")

# 3) 벡터스토어 (영속 디렉토리 쓰면 런타임 재시작해도 유지)
persist_dir = "/content/chroma_pdf_db"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 비용 절약형
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_dir
)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
print("✅ 인덱싱 완료")


총 문서 조각(페이지 기준): 30
총 청크 수: 52
✅ 인덱싱 완료


## 2-5 RA<U>Generate</U> : 생성 부분 선언

PROMPT 부분은 여러분들이 원하시는대로 입력해도 좋습니다!

🤥 영어로 입력하면 모델이 더 잘 이해한대요

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

PROMPT = ChatPromptTemplate.from_template(
"""다음 컨텍스트(업로드한 PDF에서 추출됨)만을 바탕으로 질문에 한국어로 답하라.
불확실하면 모른다고 말하라. 핵심 근거를 요약하고, 각 근거 옆에 출처(파일/페이지)를 표시하라.

[질문]
{question}

[컨텍스트]
{context}
"""
)

rag_chain = (PROMPT | llm | StrOutputParser())


## 2-6 RAG 파이프라인 langgraph로 구성

벡터db에서 검색하는 것이 <U>Retrieve</U>AG이고,

검색 결과를 생성 단계에 던져주는 것이 R<U>Augment</U>G입니다!

In [ ]:
from typing import TypedDict, List
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

# format_docs의 역할은 문서를 포맷팅해서 모델에게 제공할 'context'를 정리하는 역할입니다!
def format_docs(docs):
    # 파일명/페이지를 메타데이터에 담아두는 PyPDFLoader 기본값 사용
    lines = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("source", "")
        page = d.metadata.get("page", None)
        tag = f"{os.path.basename(src)}"
        if page is not None:
            tag += f" p.{page+1}"
        lines.append(f"[{i}] {tag}\n{d.page_content}")
    return "\n\n".join(lines)

# 그래프 선언
class QAState(TypedDict):
    question: str
    retrieved: List[Document]
    answer: str

def node_retrieve(state: QAState):
    q = state["question"]
    topk = retriever.invoke(q)
    return {"retrieved": topk}

def node_generate(state: QAState):
    q = state["question"]
    ctx = format_docs(state["retrieved"])
    ans = rag_chain.invoke({"question": q, "context": ctx})
    return {"answer": ans}

# --------- 그래프 구성 ---------
# 노드끼리 연결하기
workflow = StateGraph(QAState)
workflow.add_node("retrieve", node_retrieve) # 어떤 노드가 추가되어야 할까요?
workflow.add_node("generate", node_generate) # 어떤 노드가 추가되어야 할까요?

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate") # 어떤 노드와 어떤 노드가 연결되어야 할까요?
workflow.add_edge("generate", END)

app = workflow.compile()
print("✅ LangGraph 컴파일 완료")


✅ LangGraph 컴파일 완료


## 2-7 실제 구동해보기

question에 여러분들이 입력하고 싶은 질문을 넣어보세요!

RAG 과제는 이렇게 간단하게 끝내겠습니다😁

In [ ]:
question = "누가 나쁜 사람이야?"
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
나쁜 사람에 대한 명확한 언급은 없지만, 심청전의 맥락에서 심청이의 아버지인 심봉사와 그의 주변 인물들이 겪는 고통과 어려움이 드러납니다. 심봉사는 눈이 멀고, 딸 심청이의 희생을 통해 자신의 눈을 뜨려는 상황에서, 심청이의 효심이 강조됩니다. 

1. 심청이의 아버지 심봉사는 눈이 멀고, 딸이 동냥으로 생계를 이어가고 있어 매우 힘든 상황에 처해 있다. (출처: p.9)
2. 심청이는 아버지를 위해 자신의 몸을 팔겠다고 결심하며, 이는 그녀의 효심을 보여준다. (출처: p.10)
3. 심봉사는 딸의 희생을 걱정하며 슬퍼하고, 심청이의 효성이 걱정이 된다. (출처: p.9)

결론적으로, 나쁜 사람에 대한 명확한 정의는 없지만, 심청이의 아버지 심봉사가 처한 비극적인 상황과 심청이의 희생이 중심 이야기로 보입니다.


In [ ]:
question = "각 등장인물에 대한 윤리적 판단과 극단적인 판사가 되어서 판결문을 써봐"
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
각 등장인물에 대한 윤리적 판단과 극단적인 판사가 되어 판결문을 작성해 보겠습니다.

### 1. 심청
**윤리적 판단**: 심청은 자신의 부모를 위해 자신의 목숨을 희생하는 극단적인 선택을 한 인물로, 효도와 희생정신을 상징한다. 그녀의 행동은 도덕적으로 고귀하다고 평가될 수 있다. 그러나 그녀의 선택이 개인의 생명과 행복을 포기하는 것이므로, 극단적인 상황에서의 선택으로 볼 수 있다.  
**출처**: 심청이 자신의 몸을 팔아 인당수에 몸을 던진 사건 (p.30)

### 2. 심봉사
**윤리적 판단**: 심봉사는 가난한 상황에서도 정직하고 청렴한 삶을 살며, 가족을 위해 최선을 다하는 인물이다. 그러나 그의 무능력함이 가족에게 고통을 주는 결과를 초래하기도 한다. 그는 도덕적으로 긍정적인 인물이나, 현실적인 문제 해결 능력 부족으로 비판받을 수 있다.  
**출처**: 심봉사의 가난한 상황과 그의 성품에 대한 설명 (p.1)

### 3. 곽씨 부인
**윤리적 판단**: 곽씨 부인은 가정의 생계를 위해 힘든 일을 마다하지 않고 헌신하는 인물로, 그녀의 노력은 칭찬받아야 한다. 그러나 그녀의 희생이 가족의 행복을 보장하지 못하는 현실은 비극적이다. 그녀는 도덕적으로 훌륭하지만, 사회적 구조의 문제로 인해 고통받는 인물이다.  
**출처**: 곽씨 부인의 헌신적인 노력과 가정에 대한 설명 (p.1)

### 판결문
**사건명**: 심청의 희생과 가족의 고난에 대한 판결

**판결문**:
본 법원은 심청의 사건을 심리한 결과, 심청은 자신의 부모를 위해 극단적인 선택을 한 점에서 효도와 희생정신을 보여주었으나, 그 선택이 개인의 생명과 행복을 포기하는 것이었음을 감안하여, 그녀의 행동은 도덕적으로 고귀하나 비극적임을 인정한다. 

심봉사와 곽씨 부인의 경우, 그들은 가난 속에서도 가족을 위해 최선을 다한 점에서 긍정적으로 평가되나, 그들의 무능력함이 가족에게 고통을 주는 결과를 초래한 점은 비판받아야 한다. 

따라서, 본 법원은 심청의 희생을 기리며

In [ ]:
question = "심봉사와 곽씨 부인에게 형량을 부여해. 실제 형량 부여. 벌금형, 집행유예, 징역형 등"
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
심봉사와 곽씨 부인에게 형량을 부여하기 위해서는 그들의 행동과 상황을 고려해야 합니다. 

1. **심봉사**: 심봉사는 자신의 딸 심청을 인당수에 빠뜨려서 눈을 뜨게 하려는 극단적인 선택을 하였습니다. 이는 도덕적으로 비난받을 수 있는 행동입니다. 그러나 심봉사는 자신의 처지를 극복하기 위해 노력하는 인물로 묘사되며, 그의 행동은 딸에 대한 사랑에서 비롯된 것입니다. 따라서 형량을 부여한다면, **집행유예**를 고려할 수 있습니다. 이는 그의 행동이 비록 극단적이지만, 그 배경에 사랑과 절박함이 있었기 때문입니다. (출처: doc_1_심청전_현대문.pdf p.24)

2. **곽씨 부인**: 곽씨 부인은 가난한 상황에서도 가족을 위해 열심히 일하며, 도덕적으로 올바른 행동을 지속해왔습니다. 그녀는 품팔이를 하며 가정을 지키기 위해 노력했으며, 이는 긍정적인 평가를 받을 만한 행동입니다. 따라서 그녀에게는 형량을 부여할 필요가 없으며, 오히려 **상**을 주는 것이 적절할 것입니다. (출처: doc_1_심청전_현대문.pdf p.1)

결론적으로, 심봉사에게는 집행유예, 곽씨 부인에게는 상을 주는 것이 적절하다고 판단됩니다.


# 🔍 3. sLM

Copyright 2024 Google LLC.

## 3-1. sLM(Gemma) 로드
### sLM(small Language Model)의 대표 모델인 Gemma를 로드 해봅시다!

### (1) 사전 준비 사항
### 1. HuggingFace 로그인

https://huggingface.co/

이 링크로 들어가서 HuggingFace에 로그인합니다.(계정이 없다면 회원가입 해주세요!)

### 2. 토큰 발급받기

https://huggingface.co/settings/tokens

- 이 링크로 들어가서 'Create new token'을 클릭
- Token type: 'Read'로 해주세요.
- Token name: 아무거나 상관 없습니다.
- (⭐ 매우 중요!) 토큰 만드시고 **반드시 토큰 문자열 (hf_... 로 시작함)을 복사해주세요!**  
한 번만 볼 수 있습니다!

### 3. google colab에서 토큰 등록하기
- 왼쪽에 열쇠 모양 버튼(보안 비밀)을 눌러주세요.
- 이름: colab-gemma
- 값: 복사해둔 토큰 문자열(hf_...로 시작함)
- 노트북 액세스 : 스위치 키기(파란색)

### 4. gemma-2b-it 모델 사용 약관 동의

https://huggingface.co/google/gemma-2b-it

- (⭐ 매우 중요!) 반드시 2번 단계에서 토큰을 발급받았던 바로 그 Hugging Face 계정으로 로그인된 상태에서 위 링크로 들어가야 합니다.
- 링크로 들어가서 'Access Gemma on Hugging Face' 부분의 약관을 읽고 동의 버튼을 클릭해 주세요.
- "You have been granted access to this model"이라는 메시지가 뜨면 성공입니다.

### (2) 필요한 툴 설치

In [1]:
!pip install -U bitsandbytes
!pip install --upgrade -q transformers huggingface_hub peft \
  accelerate bitsandbytes datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.3 MB/s eta 0:00:00


In [2]:
# HuggingFace에 로그인합니다.

from huggingface_hub import login
from google.colab import userdata

# 따음표 안에 본인의 보안 비밀 키의 이름을 적습니다.(여기선 colab-gemma)
try:
    login(token=userdata.get("colab-gemma"))
except Exception as e:
    print(f"Error logging in to Hugging Face: {e}")
    print("Please make sure you have added your Hugging Face token to Colab secrets with the name 'colab-gemma'")

In [3]:
# 모델을 로드합니다.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 계산 부하를 줄이고 추론 속도를 높이기 위해 '양자화'합니다.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# sLM(gemma)를 Q&A, 요약 등에 활용하기 위해 '지시 튜닝(Instruction-Tuned) 버전의 모델을 사용합니다.
# "google/gemma-2b-it" 모델을 사용합니다.
model_id ="google/gemma-2b-it"

# 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델을 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # 양자화(quantization)
    device_map={"":"cuda:0"} # Changed from "0" to "cuda:0"
)

# GPU를 사용하도록 설정합니다.
device = "cuda"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## 3-2. sLM 활용
- sLM은 LLM을 경량화한 모델이기 때문에 언어 모델의 기능인 질의응답(추론), 요약, 분류(감정분석) 등이 가능합니다.
- **속도는 비교적 빠르지만, 성능이 그닥 좋지 않다는 것을 확인하실 수 있습니다.**

### Gemma 활용 예시 (Common Use Cases)


### (1) 질의응답(Reasoning)

#### 원하는 질문을 한 번 넣어보세요!

In [5]:
print("--- 예시 1: 질의응답 (Reasoning) ---")

# content : '~~' 따음표 안에 질문을 써주시면 됩니다.
# 하고 싶은 질문 아무거나 넣어보세요!

# "role": "user" 입니다.

chat = [
    # Gemma 모델의 채팅 형식에 맞춰 'role'과 'content'를 지정합니다.
    { "role": "user", "content": "심청전의 등장인물에 대해서 현대적으로 법률적 판단과 판결을 내려줘" }
]

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성)
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)

--- 예시 1: 질의응답 (Reasoning) ---
심청전은 한국의 대표적인 법률적 판단과 판결을 내리는 기관으로, 법원과 법무부가 주장하는 사건의 진위와 정당성을 평가하고, 이를 바탕으로 법률적 판단을 내리는 역할을 한다.

**심청전의 역할**

* **사건 진위 평가:** 심청전은 법원이 주장하는 사건의 진위와 정당성을 평가한다.
* **법률적 판단:** 심청전은 사건에 대한 법률적 판단을 내리기 위해 법원의 판결을 검토한다.
* **법무부 고시:** 심청전은 법무부가 법률적으로 문제가 있는 사건을 고시할 수 있도록 지원한다.
* **법률 정보 제공:** 심청전은 법률 정보를 제공하는 역할을 수행한다.

**현대적인 법률적 판단과 판결**

* 심청전은 1963년 법원법 제34조에 명시되어 있으며, 사건 진위와 정당성을 평가


### (2) 요약 (Summarization)

#### 원하는 글을 넣어서 요약해 보세요!

In [6]:
print("\n--- 예시 2: 요약 (Summarization) ---")

# text_to_summarize = """~~~""" 따음표 안에 요약하고 싶은 글을 넣어주시면 됩니다.""
# 요약하고 싶은 글 아무거나 넣어보세요!

text_to_summarize = """
네이버와 두나무의 합병 절차가 본격화되고 있다. 오는 26일 양사 이사회에서 포괄적 주식교환 안건이 상정될 예정이어서 합병이 사실상 확정 단계에 들어갔다는 관측이 나온다.

19일 정보통신기술(ICT) 업계에 따르면 네이버와 가상자산 거래소 업비트를 운영하는 두나무는 다음주 이사회에서 주식교환 방식을 논의한다. 교환 비율은 네이버 1주당 두나무 3~4주 수준이 유력하게 거론되고 있다. 구체적인 비율은 이사회 직전 윤곽이 드러날 전망이다.

합병이 성사되려면 이사회 의결 이후 주주총회 특별결의를 통과해야 한다. 출석 주주의 3분의 2 이상, 발행주식 총수의 3분의 1 이상이 동의해야 한다는 규정을 충족해야 한다.

주식교환이 마무리되면 네이버의 금융 계열사인 네이버파이낸셜과 두나무가 상호 지분을 교환하게 된다. 이로써 두나무는 네이버파이낸셜의 자회사로 편입되고, 네이버 입장에서는 두나무가 손자회사가 되는 구조다.

합병 이후 두나무를 품은 네이버파이낸셜의 최대 주주는 송치형 두나무 회장이, 2대 주주는 네이버가 될 것으로 예상된다.

한때 업계에서는 이번 합병이 금융과 가상자산 사업의 분리를 규정하는 ‘금가분리’ 원칙에 저촉될 수 있다는 우려도 제기됐다. 그러나 금융당국이 해당 구조가 규제 위반에 해당하지 않는다고 판단한 것으로 알려지면서, 합병 추진이 한층 속도를 내고 있다.

"""

# content : '~~~'에 요청사항을 적으시면 됩니다. 여기서는 위 글을 세 문장으로 요약해 달라고 요청했습니다.
# "role": "user" 입니다.

chat = [
    { "role": "user", "content": f"Could you summarize the following text in three sentence?\n\nText:\n{text_to_summarize}" }
]

# 여기부터는 각 사례 모두 코드가 동일합니다.

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성).
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)


--- 예시 2: 요약 (Summarization) ---
1. Nuera and Donaut are merging their stock exchange operations.
2. The merger is expected to be finalized after the upcoming shareholder meeting.
3. The combined company will be listed on the stock exchange, with Nuera becoming the controlling shareholder.


### (3) 분류 (Classification)

원하는 문장을 넣고 감정 분석을 해보세요!

In [8]:
print("\n--- 예시 3: 분류 (Classification) ---")

# content: '~~~' 따음표 안에 요청사항을 적으시면 됩니다.
# 여기서는 긍정, 부정, 중립으로 감정분석을 요청했습니다.

# \n\nText: 여기에 감정분석할 문장을 적어주시면 됩니다.
# 원하는 문장을 넣고 감정 분석을 해보세요!

# "role": "user" 입니다.

chat = [
    { "role": "user", "content": "Classify the text into neutral, negative, or positive. Generate only the class, nothing else.\n\nText: 때낀 수족관 닦는 기분" }
]

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성).
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)


--- 예시 3: 분류 (Classification) ---
Neutral.

The text is describing a neutral activity, the act of washing one's feet in a bath or shower.



#### sLM 활용사례 결과 요약

- sLM은 질의응답이나 요약은 어느 정도 수행하지만, 퀄리티가 아쉽습니다.
- 감정 분석에서 '너무 재미가 없다'를 중립으로 판단하는 것으로 보아 분류 작업에서도 한계가 나타납니다.


- 이렇게 범용적인 활용에서는 한계가 나타나지만, 미세조정(Fine-Tuning)을 통해 특정 작업에 특화하면 해당 작업에는 좋은 성능을 내면서도 여전히 경량화된 모델의 장점을 누릴 수 있는 것이 sLM의 특징입니다!

# 🤩🤩 수고하셨습니다 🤩🤩
⭐️⭐️ 2-2의 API KEY 셀을 지우고 과제 제출해주세요!! ⭐️⭐️